In [1]:
# Import statements
from hashlib import sha1

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import openpyxl
import pylab as plt
import glob
import os

import nltk

plt.rcParams["font.size"] = 16

from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier

from sklearn.compose import make_column_transformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.pipeline import make_pipeline
from wordcloud import WordCloud
from yellowbrick.cluster import KElbowVisualizer
from yellowbrick.cluster import SilhouetteVisualizer

from sentence_transformers import SentenceTransformer
import torch

import regex as re
import pandas as pd
from bs4 import BeautifulSoup

import io

import requests
from pypdf import PdfReader

# UBC Salaries Across Gender, Department, and Job Title

## Introduction

The primary objective of this project is to develop skills in data wrangling, machine learning, and Tableau. A secondary goal is to create a Tableau dashboard for visualizing University of British Columbia (UBC) faculty salaries based on gender, department, and job title.

To prioritize learning, rather than directly requesting salary data from UBC by department, job title, and gender, I will rely on the annual PDF of salary data released by UBC, along with information from UBC's faculty directory and global baby name datasets.

The project involves several steps: gathering and cleaning salary data, collecting and cleaning department and job title data, making gender predictions for each faculty member, and ultimately visualizing the cleaned data through a Tableau dashboard.

This approach incorporates tasks such as scraping, clustering, and data transformation, offering a more in-depth exploration of data manipulation techniques and fostering a comprehensive understanding of the intricacies involved in data wrangling.

## Contents

The following list is how the rest of this project will unfold:

- Data Collection and Cleaning
    - Salary Data
    - Department and Job Title Data
- Gender Prediction
- Data Visualization
- Conclusion

## Data Collection and Cleaning

### Salary Data

We can use the PDF salary information that UBC releases every year. The salary information is retrieved from the following website: [UBC Financial Reports](https://finance.ubc.ca/reporting-planning-analysis/financial-reports). The first step is to go to this website and find the links for years with available salary data.

In [2]:
links = {} # get all financial report links
r = requests.get('https://finance.ubc.ca/reporting-planning-analysis/financial-reports') # go to financial report webpage
soup = BeautifulSoup(r.content, 'html.parser')
finance_section = soup.find('h3', string = re.compile('Financial Information Act')) # find the "Financial Information Act" header
salary_link_elements = finance_section.find_next_sibling('ul').findAll('li') # find the section with the links
for element in salary_link_elements: # collect all of the links
    salary_link = element.find('a').get('href') # get link from element
    year = "20" + re.search("FY([0-9][0-9])", salary_link).group(1) # use regex to get the year in each link
    links[year] = salary_link # add year and link to dictionary 
links

{'2023': 'https://finance.ubc.ca/sites/finserv.ubc.ca/files/FY23%20UBC%20Statement%20of%20Financial%20Information.pdf',
 '2022': 'https://finance.ubc.ca/sites/finserv.ubc.ca/files/FY22%20UBC%20Statement%20of%20Financial%20Information.pdf',
 '2021': 'https://finance.ubc.ca/sites/finserv.ubc.ca/files/FY21%20UBC%20Statement%20of%20Financial%20Information.pdf',
 '2020': 'https://finance.ubc.ca/sites/finserv.ubc.ca/files/FY20_Financial_Information_Act_Report.pdf'}

Above we can see the years with available salary information.

Next, I will create a function that when given a link, will return the text from the link in string form.

In [3]:
def fetch_salary_data(link):
    '''find UBC salary pdf for given year and return it in string form'''
    url = link
    
    r = requests.get(url)
    f = io.BytesIO(r.content)
    
    all_text = "" # variable to store all text in pdf
    reader = PdfReader(f)
    contents = reader.pages # returns a list of pages
    for content in contents: # for each page, extract text
        all_text += content.extract_text() # add text to all_text
        
    return all_text

Now, I will create a function that can take the salary data in string form, clean it up, and return a dataframe with the columns: `First Name`, `Last Name`, `Remuneration`, `Expenses`, and `Year`.

In [4]:
def clean_salary_data(raw_data, year):
    '''take salary data in string form and turn it into a dataframe, add a column and fill it with the given year'''
    # Remove beginning/end text
    raw_data_a = raw_data.split('Earnings greater than')[0].split('external cost recoveries.')[1]
    raw_data_a
    
    # Remove unnessessary lines
    raw_data_b = re.split('([\.\p{L},\s-]+[\s\n]+[0-9,-]+[\s\n]+[0-9,-]+)', raw_data_a)
    
    raw_data_c = [i for i in raw_data_b if (',' in i) and ('SCHEDULE' not in i)]
    
    # Remove spaces and new lines
    raw_data_d = [i.replace('\n',' ').replace("  "," ").strip() for i in raw_data_c]
    raw_data_d = [i.replace("  "," ") for i in raw_data_d]
    
    # # Split data into Names/Remuneration/Expenses
    raw_data_e = [i.rsplit(' ',2) for i in raw_data_d]
    raw_data_e
    
    # Create Column names
    ubc_salary_data = pd.DataFrame(raw_data_e, columns = ['Name', 'Remuneration', 'Expenses'])
    
    # Split Name into First/Last Name
    ubc_salary_data['First Name'] = ubc_salary_data['Name'].str.split(', ', expand = True)[1]
    ubc_salary_data['Last Name'] = ubc_salary_data['Name'].str.split(', ', expand = True)[0]
    
    
    # Select necessary columns
    ubc_salary_data = ubc_salary_data[['Last Name','First Name','Remuneration','Expenses']]
    
    # turn salary column from string to numeric
    ubc_salary_data['Remuneration'] = ubc_salary_data['Remuneration'].astype(str).str.replace(',','')
    ubc_salary_data['Remuneration'] = pd.to_numeric(ubc_salary_data['Remuneration'], errors='coerce')

    ubc_salary_data['Year'] = f"{year}"

    return ubc_salary_data

Now I will loop through the salary data links I collected, clean up the data for each year, and combine the data into one dataframe. 

In [28]:
salary_data = pd.DataFrame(columns = ['Last Name', 'First Name', 'Remuneration', 'Expenses','Year']) # create empty dataframe for salary data
most_recent = True # keeps track of most recent salary data
for year, link in links.items(): # for each year that UBC has data for
    raw = fetch_salary_data(link) # get raw data in string form
    salaries = clean_salary_data(raw, year) # get clean data as a dataframe
    if most_recent: # export the most recent salary data so that we can collect the new department/job title info
        salaries.to_csv(f"/Users/jadebouchard/Desktop/UBC Salaries/Salary Data/{year}.csv")
        most_recent = False
    if salary_data.empty: # avoid warning that we shouldn't be concatenating empty dataframes
        salary_data = salaries
    else:
        salary_data = pd.concat([salary_data,salaries]) # paste dataframes together

Below you can see a recent and less recent section of the data.

In [6]:
salary_data.head()

,Last Name,First Name,Remuneration,Expenses,Year
0,Aamodt,Tor,193153.0,"5,597",2023
1,Abanto Salguero,Arleni Karina,107723.0,393,2023
2,Abbassi,Arash,109136.0,82,2023
3,Abdalkhani,Arman,101829.0,-,2023
4,Abdi,Ali,238203.0,"2,981",2023


In [7]:
salary_data.tail()

,Last Name,First Name,Remuneration,Expenses,Year
6219,Zumbo,Bruno,294953.0,"10,102",2020
6220,Zumpano,Franco,94463.0,420,2020
6221,Zumrawi,Abdel Azim,98145.0,-,2020
6222,Zwicker,Jill,145343.0,"12,523",2020
6223,Zysk,Eva,93058.0,"2,489",2020


This scraping process missed about 70 entries due to formatting issues. These entries will be removed going forward.

In [8]:
salary_data = salary_data[~salary_data.isnull().any(axis = 1)]

Also, now that the department and title data has been merged onto the salary data, to try and maintain consistency over the years names will be shortened. For example, "A Bobby" will be shortened to "Bobby" and "Anne Michele" will be shortened to "Anne".

In [9]:
def shorten_name(name):
    '''remove initials from names, and if two names just keep the first one'''
    name = name.strip() # remove white space
    name = name.replace(" -","-").replace("- ","-") # make sure names that should be connected are connected
    if len(name.split(" ")) <= 1: # if only one word, return word
        return name
    else:
        words = name.split(" ")
        for word in words: # remove initials if they exist
            if len(word) == 1:
                words.remove(word)
        return words[0] # return first word

salary_data.loc[:,"First Name"] = salary_data["First Name"].apply(shorten_name)
salary_data.loc[:,"Last Name"] = salary_data["Last Name"].apply(shorten_name)

Now we have clean salary data for UBC staff members.

In [10]:
salary_data.to_excel('data/UBC_Salary_DF_All_Years.xlsx', index = False)
salary_data.to_csv('data/UBC_Salary_DF_All_Years.csv', index = False, encoding = "utf8")